En el último capítulo vimos que las esquinas son regiones de la imagen con una gran variación de intensidad en todas las direcciones. Chris Harris y Mike Stephens realizaron uno de los primeros intentos de encontrar estas esquinas en su artículo A Combined Corner and Edge Detector en 1988, por lo que ahora se llama Harris Corner Detector. Llevó esta simple idea a una forma matemática. Básicamente encuentra la diferencia de intensidad para un desplazamiento de( u , v )en todas direcciones. Esto se expresa de la siguiente manera:

Detector de esquinas Harris en OpenCV
OpenCV tiene la función cv.cornerHarris() para este propósito. Sus argumentos son:

img : imagen de entrada. Debe ser en escala de grises y tipo float32.
blockSize : es el tamaño del vecindario considerado para la detección de esquinas.
ksize : parámetro de apertura de la derivada de Sobel utilizada.
k - Parámetro libre del detector de Harris en la ecuación.

In [1]:
import numpy as np
import cv2 as cv
filename = 'resources/chessboard.png'
img = cv.imread(filename)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
gray = np.float32(gray)
dst = cv.cornerHarris(gray,2,3,0.04)
#result is dilated for marking the corners, not important
dst = cv.dilate(dst,None)
# Threshold for an optimal value, it may vary depending on the image.
img[dst>0.01*dst.max()]=[0,0,255]
cv.imshow('dst',img)
if cv.waitKey(0) & 0xff == 27:
    cv.destroyAllWindows()

Esquina con precisión de subpíxeles.

A veces, es posible que necesites encontrar las esquinas con la máxima precisión. OpenCV viene con una función cv.cornerSubPix() que refina aún más las esquinas detectadas con una precisión de subpíxeles. A continuación se muestra un ejemplo. Como de costumbre, primero debemos encontrar las esquinas de Harris. Luego pasamos los centroides de estas esquinas (puede haber un montón de píxeles en una esquina, tomamos su centroide) para refinarlos. Las esquinas Harris están marcadas con píxeles rojos y las esquinas refinadas están marcadas con píxeles verdes. Para esta función, tenemos que definir los criterios para detener la iteración. Lo detenemos después de un número específico de iteraciones o de que se alcance una cierta precisión, lo que ocurra primero. También necesitamos definir el tamaño del barrio en el que busca esquinas.

In [2]:
import numpy as np
import cv2 as cv
filename = 'resources/chessboard.png'
img = cv.imread(filename)
gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
# find Harris corners
gray = np.float32(gray)
dst = cv.cornerHarris(gray,2,3,0.04)
dst = cv.dilate(dst,None)
ret, dst = cv.threshold(dst,0.01*dst.max(),255,0)
dst = np.uint8(dst)
# find centroids
ret, labels, stats, centroids = cv.connectedComponentsWithStats(dst)
# define the criteria to stop and refine the corners
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.001)
corners = cv.cornerSubPix(gray,np.float32(centroids),(5,5),(-1,-1),criteria)
# Now draw them
res = np.hstack((centroids,corners))
res = np.int0(res)
img[res[:,1],res[:,0]]=[0,0,255]
img[res[:,3],res[:,2]] = [0,255,0]
cv.imwrite('resources/subpixel5.png',img)

True